In [1]:
%pip install geoviews -q
%pip install cartopy -q
%pip install jupyter_bokeh -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# OS packages
import os
from datetime import datetime, time
import pytz

# Data handling
import pandas as pd
import numpy as np


# Geospatial
import geopandas as gpd
import cartopy.crs as ccrs

# Visualisation
import holoviews as hv
import geoviews as gv
from holoviews import opts
import hvplot.pandas
import hvplot.xarray 
import panel as pn

import plotly.express as px

hv.extension('bokeh', 'plotly', 'matplotlib')
pn.extension(sizing_mode="stretch_width", template="bootstrap", theme="default")

In [3]:
input_practice_dir = '/workspace/notebooks/sandbox/data/input-data/Practice information/'
input_geom_dir = '/workspace/notebooks/sandbox/data/input-data/Stratification/'
output_dir = '/workspace/notebooks/sandbox/data/input-data/Practice information/practice-information.csv'


practice_csv = pd.read_csv(os.path.join(input_practice_dir, 'practice-information.csv'), usecols=['label','Year', 'Land use', 'Urea', 'Lime','Synthetic_fertiliser','Biochar'])
practice_csv=practice_csv[practice_csv['label']>0]

#use this version of the csv for the dropdown menu select:
practice_csv_drop_year= practice_csv.drop(['Year'], axis=1)

geom_gdf = gpd.read_file(os.path.join(input_geom_dir, '1/cea part/cea_1.shp'))
geom_gdf.to_crs("WGS84", inplace=True)
geom2_gdf = gpd.read_file(os.path.join(input_geom_dir, '2/cea part/cea_2.shp'))
geom2_gdf.to_crs("WGS84", inplace=True)
geom3_gdf = gpd.read_file(os.path.join(input_geom_dir, '3/cea part/cea_3.shp'))
geom3_gdf.to_crs("WGS84", inplace=True)


In [4]:
practice_csv.head()

,label,Year,Land use,Lime,Urea,Biochar,Synthetic_fertiliser
6,1,2022,Crop,Y,Y,N,Y
7,1,2021,Crop,N,Y,N,Y
8,1,2020,Crop,N,Y,N,Y
9,1,2019,Crop,N,Y,N,Y
10,1,2018,Crop,N,Y,N,Y


In [5]:
geom_gdf

,FID,geometry
0,0,"POLYGON ((118.21607 -33.66328, 118.22050 -33.6..."
1,1,"POLYGON ((118.22927 -33.66327, 118.22927 -33.6..."


in the terrawise data, 3 shapefiles are provided. for cea 1 there are 2 polygons, and for the others there is one each, so a total of 4 polygons across 3 shapefiles (grr). So, need to append the polygons and then make sure the FID matches the label from the csv data.

In [28]:
input_geom=[geom_gdf, geom2_gdf, geom3_gdf]
output_geom=[]

input_geom

[   FID                                           geometry
 0    0  POLYGON ((118.21607 -33.66328, 118.22050 -33.6...
 1    1  POLYGON ((118.22927 -33.66327, 118.22927 -33.6...,
    FID                                           geometry
 0    0  POLYGON ((118.20219 -33.72547, 118.20211 -33.7...,
    FID                                           geometry
 0    0  POLYGON ((118.21056 -33.73780, 118.22301 -33.7...]

In [ ]:

#loop to attach practice data to each CEA
for i,geom in enumerate (input_geom):
    #print("STARTING GEOM: \n",geom)
    cea=i+1  #ideally this is programmatic, but for now setting as 1 so index starts right
    #frame = geom.dissolve() #cant do this as there are actual multiple polygons.
    frame['label']=cea
    frame_pracice_data = practice_csv[practice_csv['label']==cea]
    geom_combined = frame.set_index('label').join(frame_pracice_data.set_index('label'))
    output_geom.append(geom_combined)
    
#print(output_geom[0].head())


In [13]:
map_view_1 = geom_gdf.hvplot(geo=True, tiles='ESRI', line_width=3, fill_alpha=0, frame_width=600, c='FID')
map_view_1

AttributeError: 'list' object has no attribute 'hvplot'

In [16]:
cea1 = output_geom[0]
cea2 = output_geom[1]
cea3 = output_geom[2]

In [21]:
cea1.hvplot(geo=True, tiles='ESRI', line_width=3, fill_alpha=0, frame_width=600, c='Year')

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Year)

In [23]:
outpath = '/workspace/notebooks/sandbox/data/output-data/spatial-joining'
fname = 'cea1.geojson'

fname_path = outpath + '/' + fname 
cea1.to_file(fname_path, driver='GeoJSON')